<a href="https://colab.research.google.com/github/quanvu0996/ds_test/blob/main/DS_TEST_PRACTICE_T3_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ĐÁNH GIÁ NĂNG LỰC KỸ SƯ KHOA HỌC DỮ LIỆU THÁNG 03 - 2022
#### BÀI THỰC HÀNH
---

Ngày test: 25/03/2022
Thời gian làm bài: 85 phút, từ 10:00 đến 11:25
(+ 5 phút thiết lập môi trường và nộp bài)


**Quy định:**
- Không trao đổi trong suốt quá trình làm bài
- Record màn hình làm bài và share lại cho quanvh8 sau khi kết thúc thời gian test
- Được sử dụng tài liệu và tra cứu internet
- Sau khi hoàn thành, tải notebook, gửi về email: quanvh8@viettel.com.vn

**Hướng dẫn làm bài:**

* Chạy toàn bộ lệnh trong phần 0. CHUẨN BỊ MÔI TRƯỜNG
* Làm theo hướng dẫn tại các câu và CHỈ SỬA các đoạn code nằm giữa các thẻ:

      # YOUR CODE START
      # / END YOUR CODE

*Lưu ý: hầu hết câu hỏi không có tính tuần tự (không cần hoàn thành câu trước để làm câu sau)*

*Đề thi có một số testcase hỗ trợ DS kiểm tra kết quả, nhưng đây không phải điểm số. Điểm số được giám khảo chấm và gửi lại sau.*



---

### 0. CÀI ĐẶT MÔI TRƯỜNG
Guide: Chạy toàn bộ các lệnh trong phần này <br>
KHÔNG SỬA CÁC CÂU LỆNH TRONG PHẦN NÀY

In [21]:
!pip install pyspark

In [22]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import gzip
import shutil
import requests 
import seaborn as sns
import matplotlib.pyplot as plt

import pyspark
from pyspark.sql import functions as F
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [23]:
def data_downloader(file_name):
    file_url = 'https://datasets.imdbws.com/'+file_name
    file_path = "/content/"+file_name
    
    r = requests.get(file_url, stream = True, verify=False) 
    
    with open( file_path, "wb") as file: 
        for block in r.iter_content(chunk_size = 1024*10): 
            if block: 
                file.write(block)
    print("Loaded "+file_name+" to local path: "+file_path)

# TEST CASE
class TEST():
    def test_1(df):
        tc1 = (df.columns ==['tconst', 'averageRating', 'numVotes'])
        tc2 = (df.count() >= 1227518)
        if (tc1 and tc2): print("PASS")
        else: print('FAIL')

    def test_2(df):
        tc1 = (df.columns == ['titleId','ordering','title','region','language','types','attributes','isOriginalTitle'])
        tc2 = (df.count() >= 31450282)
        if (tc1 and tc2): print("PASS")
        else: print('FAIL')

    def iia_schema(df_pred):
      tc1 = (df_pred.columns == ['tconst', 'pred'])
      tc2 = (df_pred.count() == 53956 )
      if tc1 and tc2 :
        print("PASS: dataframe đầu ra đúng định dạng")
      else:
        print("FAIL: dataframe đầu ra không đúng định dạng, đ/c kiểm tra lại")
      
      tc3 = (df_pred.join(test_ratings, on = 'tconst').count() == 53956)
      
      if tc3 :
        print("PASS: dataframe đầu ra đúng số lượng bản ghi")
      else:
        print("FAIL: dataframe đầu ra không đúng số lượng bản ghi, đ/c kiểm tra lại")
        
    def mae_iia(df_pred):
      result = df_pred.join(test_ratings, on = 'tconst')
      result = result.withColumn('ae', F.abs(F.col("pred")-F.col("averageRating")))
      result.selectExpr("avg(ae) AS mae").show()

    def iib_schema(df_pred):
      tc1 = ( list(df_pred.columns) == ['tconst', 'pred'])
      tc2 = (df_pred.count()['tconst'] == 104488 )
      if tc1 and tc2 :
        print("PASS: dataframe đầu ra đúng định dạng")
      else:
        print("FAIL: dataframe đầu ra không đúng định dạng, đ/c kiểm tra lại")
      
      tc3 = ( pd.merge(df_pred, test, how='inner', on = 'tconst')['tconst'].count() == 104488)
      
      if tc3 :
        print("PASS: dataframe đầu ra đúng số lượng bản ghi")
      else:
        print("FAIL: dataframe đầu ra không đúng số lượng bản ghi, đ/c kiểm tra lại")
        
    def score_iib(df_pred):
      eval = pd.merge(df_pred, test, how='inner', on = 'tconst' )
      print( roc_auc_score( eval['isAdult'], eval['pred'] ))
    

In [24]:
spark = pyspark.sql.SparkSession \
    .builder \
    .appName("DS_test") \
    .getOrCreate()

#### DATA: Bộ IMDb

Thông tin bộ dữ liệu IMDb: https://www.imdb.com/interfaces/

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name

![Alt text](https://relational.fit.cvut.cz/assets/img/datasets-generated/imdb_ijs.svg "Schema bộ dữ liệu")

In [25]:
# Tải dữ liệu về
file_list = ["title.akas.tsv.gz", "title.ratings.tsv.gz", 
             "title.basics.tsv.gz", "title.basics.tsv.gz", "title.crew.tsv.gz", "title.principals.tsv.gz",
            "name.basics.tsv.gz" ]
      
for i in file_list:
    data_downloader(i)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Loaded title.akas.tsv.gz to local path: /content/title.akas.tsv.gz


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Loaded title.ratings.tsv.gz to local path: /content/title.ratings.tsv.gz


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

Loaded title.crew.tsv.gz to local path: /content/title.crew.tsv.gz


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Loaded title.principals.tsv.gz to local path: /content/title.principals.tsv.gz


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Loaded name.basics.tsv.gz to local path: /content/name.basics.tsv.gz


## PHẦN I. CHUNG
[ 10 điểm ] <br>
Guide: Sử dụng pyspark hoặc spark-SQL thực hiện tất cả các câu thuộc phần này.

**Câu 1**. [1đ] Đọc các file vào pyspark dataframe, biết rằng các cột trong các file dữ liệu đều phân cách nhau bởi dấu tab (\t). Lưu tương ứng các dataframe với các file dữ liệu:
- df_rating : "title.ratings.tsv.gz"
- df_akas  :"title.akas.tsv.gz"
- df_basics  :"title.basics.tsv.gz"
- df_crew  :"title.crew.tsv.gz"
- df_principals :"title.principals.tsv.gz"
- df_person  : "name.basics.tsv.gz"

(Xem mô tả quan trọng của dữ liệu phía trên)

In [26]:
df_ratings = (
    # YOUR CODE START 
        
    # / END YOUR CODE
)

df_akas= (
    # YOUR CODE START 
        
    # / END YOUR CODE
)

df_basics = (
    # YOUR CODE START 
        
    # / END YOUR CODE
)

df_crew = (
    # YOUR CODE START 
        
    # / END YOUR CODE
)

df_principals = (
    # YOUR CODE START 
        
    # / END YOUR CODE
)

df_person = (
    # YOUR CODE START 
        
    # / END YOUR CODE
)

In [28]:
TEST.test_1(df_ratings)
TEST.test_2(df_akas)

**Câu 2**. [1đ] Tìm top 3 ngôn ngữ có nhiều nội dung ( bao gồm tất cả nội dung movie, tv show,...) nhất trong bộ dataset này

In [ ]:
# YOUR CODE START 

# / END YOUR CODE

**Câu 3**. [2đ]
Trong các bộ phim Mỹ (region="US", titleType="movie") được công chiếu giai đoạn 2010-2015, tìm top 5 thể loại phim có bình quân rating điểm cao nhất.

In [ ]:
# YOUR CODE START 

# / END YOUR CODE

**Câu 4**. [3đ] Bạn đặt ra một giả thuyết rằng một đạo diễn có xu hướng hợp tác với các diễn viên thân thuộc với họ, hơn là liên tục tìm kiếm các diễn viên mới cho các phim khác nhau. <br>
Hãy lựa chọn các metrics phù hợp, phân tích từ bộ dữ liệu thực tế và rút ra kết luận cho giả thuyết trên có đúng hay không? <br>
( Phân tích trên thể loại phim titleType="movie", region="US",  thời gian công chiếu trong khoảng 2000-2020)

In [ ]:
# YOUR CODE START 

# / END YOUR CODE

In [ ]:
your_conclusion = ('''
# YOUR CONCLUSION HERE


''')

**Câu 5**. [3đ] Một giải thưởng về phim (titleType = 'movie') được trao cho các đạo diễn. Danh sách đề cử là các đạo diễn thỏa mãn các điều kiện: <br>
1. Đạo diễn vẫn còn sống
2. Có trên 5 bộ phim hay (phim hay là phim được chấm điểm averageRating >7.5).
3. Tỷ trọng phim tệ (phim tệ là phim có averageRating <= 4) chiếm không quá 20% tổng số phim của đạo diễn đó và không quá 10 bộ phim.

Trong đó, averageRating chỉ được tính khi có trên 200 phiếu bầu. <br>
Giải thưởng được trao cho 3 trong số các đạo diễn thuộc danh sách đề cử. <br>
Đặt p (float) là xác suất để đạo diễn trúng cử đồng thời có kiêm biên kịch (writer, xác định bởi primaryProfession).  <br>
Hãy tính p.

In [ ]:
# YOUR CODE START 


# / END YOUR CODE

### PHẦN II. TỰ CHỌN
[ 10 điểm ] <br>
DS chọn 1 trong 2 câu II.a hoặc II.b
**Chỉ sử dụng dữ liệu train đã được chia sẵn để huấn luyện model, không sử dụng tập test.** <br>
Có thể sử dụng tập test để đánh giá độ chính xác của model.

Điểm số được chấm dựa trên độ chính xác của model trên tập test.

**Câu II.a**. Trên môi trường pyspark, hãy tổng hợp dữ liệu và huấn luyện một mô hình dùng pyspark MLlib dự đoán averageRating

In [ ]:
# CHẠY ĐOẠN CODE SAU | DO NOT MODIFY
train_film = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")>=2000) & (F.col("startYear")<=2015) ).select("tconst")
test_film  = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")> 2015) & (F.col("startYear")<=2021) ).select("tconst")

train_ratings = df_ratings.join( train_film, on = 'tconst')
test_ratings = df_ratings.join( test_film, on = 'tconst')

In [ ]:
train_ratings.count()

Chỉ sử dụng thông tin của các bộ phim trong train_film (và nhãn là averageRating trong train_ratings) để xây dựng model. <br>
Có thể join thêm với các thông tin ở các bảng khác không giới hạn để bổ sung feature. <br>
Metric đánh giá là MAE

**YÊU CẦU ĐẦU RA** <br>
Dataframe: df_pred {tconst: string, pred: float}, trong đó tconst là các mã bộ phim tương ứng với các bộ phim trong test_film, pred là dự đoán averageRating của model.

In [ ]:
# YOUR CODE START 

#...BUILD YOUR MODEL...

df_pred = 
# / END YOUR CODE

In [ ]:
# df_pred hợp lệ khi testcase báo 2 thông báo PASS như sau:
#>> PASS: dataframe đầu ra đúng định dạng
#>> PASS: dataframe đầu ra đúng số lượng bản ghi
TEST.iia_schema( df_pred )

In [ ]:
# Tính MAE
TEST.mae_iia( df_pred )

**Câu II.b**. Bạn đưa ra giả thuyết thông tin về tiêu đề phim có thể giúp phân loại phim đó có phải là phim không dành cho trẻ nhỏ hay không. <br>
Chỉ sử dụng thông tin từ bảng title.basics.tsv.gz, hãy xây dựng mô hình dự đoán nhãn isAdult. Trong đó, bắt buộc có sử dụng xử lý chuỗi thông tin tiêu đề phim originalTitle.

(Mô hình có thể được xây dựng bằng pyspark hoặc python)

In [ ]:
# CHẠY ĐOẠN CODE SAU | DO NOT MODIFY
train = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")>=2000) & (F.col("startYear")<=2015) ).toPandas()
test = df_basics.filter((F.col("titleType")=="movie") & (F.col("startYear")> 2015) & (F.col("startYear")<=2021) ).toPandas()

In [ ]:
train['tconst'].count()

Có thể sử dụng model pyspark hoặc python. <br>
Metric đánh giá là accuracy

**YÊU CẦU ĐẦU RA** <br>
- Mô hình: có các bước biến đổi dữ liệu và xử lý text
- Dữ liệu: pandas dataframe df_pred {tconst: string, pred: int}, trong đó tconst là các mã bộ phim tương ứng với các bộ phim trong test, pred là dự đoán isAdult của model, pred phải mang giá trị 0 hoặc 1.

In [ ]:
# YOUR CODE START 

df_pred = 
# / END YOUR CODE

In [ ]:
# df_pred hợp lệ khi testcase báo 2 thông báo PASS như sau:
#>> PASS: dataframe đầu ra đúng định dạng
#>> PASS: dataframe đầu ra đúng số lượng bản ghi
TEST.iib_schema(df_pred)

In [ ]:
# Tính AUC
TEST.score_iib(df_pred)

**--- YOUR TEST END HERE ---**